# Visualizing FastText Model

This notebook develops functions for visualizing semantic relationships preserved in a FastText model of the Sumerian corpus using t-SNE and Bokeh.

Inspired by [*LDA visualized using t-SNE and Bokeh*](https://www.kaggle.com/yohanb/lda-visualized-using-t-sne-and-bokeh) by Yohan, and [*Visualizing Word Vectors with t-SNE*](https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne) by Jeff Delayney.

Note: for visualization Bokeh is preferred over Matplotlib primarily because of the difficulties in using a custom font in Matplotlib. This becomes a major obstacle when trying to represent tokens in cuneiform.

In [256]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.filterwarnings(action='ignore', category=FutureWarning, module='gensim' )
import gensim
import numpy as np
import pickle
from sklearn.manifold import TSNE
from gensim.models.fasttext import FastText as FT_gensim
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import ColumnDataSource, LabelSet, Legend, LegendItem, Range1d #, HoverTool, CustomJS, , Slider
from bokeh.layouts import column
from bokeh.palettes import all_palettes
from bokeh.palettes import Category20
output_notebook()
# note: more recent bokeh versions require notebook 5 or JupyterLab

Loading BokehJS ...

# Load the Models
There are three models: 
- model_cuneiform.model: the Sumerian copus in Unicode cuneiform
- model_tl.model: the Sumerian corpus in transliteration
- model_lemm.model: the Sumerian corpus in lemmatization

In [2]:
model_c = FT_gensim.load("model/model_cuneiform.model")
model_l = FT_gensim.load("model/model_lemm.model")
model_t = FT_gensim.load("model/model_tl.model")

# Cuneify
Create a function that allows input in transliteration, with output in cuneiform.

In [3]:
with open("output/ogsl.p", "rb") as p:
    o = pd.read_pickle(p)
signs_d = dict(zip(o["value"], o["utf8"]))

In [4]:
def cun(text): 
    """transform transliterated input into cuneiform. Use unicode subscript numbers and separate all signs with hyphens; separate words with blanks
    Examples: 'ma-an-gi₄'; 'd-en-lil₂ nibru-ki'.
    Transliteration style (sugal₇ vs. sukkal; dug₄ vs. du₁₁; gen vs. ŋen; etc.) and capitalization are unimportant.
    """
    cun_line = []
    words = text.lower().split()
    for word in words: 
        signs = word.lower().split('-')
        seq = [signs_d[s] if s in signs_d else s for s in signs]
        seq = ''.join(seq)
        cun_line.append(seq)
    line = ' '.join(cun_line)
    return(line)

# Create Lists of Semantically Related Words

In [174]:
def word_categories(model, words, topn=10):
    """word_categories takes an iterable with one or more words from the vocabulary of model.
    For each word a dictionary of similar words (with the target word) is construed.
    The value of each word is numerical (integer) and indicates the category to which it belongs.
    The function returns a dictionary."""
    word_d = {}
    for idx, word in enumerate(words):
        w = model.wv.most_similar(word, topn=topn)
        w = [m[0] for m in w]
        w.append(word)
        for item in w:
            if item in word_d:
                word_d[item] = len(words)
            else:
                word_d[item] = idx
    return word_d

# Project and Visualize Related Words

In [270]:
def tsne_bokeh(model, word_d, words, fontsize="12pt"):

    labels = []
    tokens = []
    categories = []
    legend = []
    
    for word in word_d:
        tokens.append(model.wv[word])
        labels.append(word)
        categories.append(word_d[word])
        if word_d[word] == len(words): 
            legend.append("mixed")
        else: 
            legend.append(words[word_d[word]])
    
    color_d = {0: "black", 1: "red", 2: "green", 3: "yellow", 4: "brown", 5: "blue"}
    colors = [color_d[category] for category in categories]
    #c = Category20[len(words) + 1]
    #colors = [c[category] for category in categories]
    
    tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    tsne_embedding = tsne.fit_transform(tokens)
    tsne_embedding = pd.DataFrame(tsne_embedding, columns=['x','y'])
    tsne_embedding["color"] = colors
    tsne_embedding["labels"] = labels
    tsne_embedding["legend"] = legend

    source = ColumnDataSource(
        data=dict(
        x = tsne_embedding.x,
        y = tsne_embedding.y,
        colors = tsne_embedding.color,
        labels=tsne_embedding.labels,
        legend = tsne_embedding.legend
        )
    )

    l = LabelSet(x='x', y='y', text='labels', level='glyph',
              x_offset=5, y_offset=5, source=source, render_mode='canvas', 
             text_font_size=fontsize, text_font="CuneiformComposite")

    plot_tsne = figure(plot_width=900, plot_height=900) #, tools=tools_tsne, title='Papers')
    plot_tsne.circle('x', 'y', size=7, fill_color='colors', 
                  line_alpha=0, line_width=0.01, source=source, legend="legend")
    plot_tsne.add_layout(l)
    plot_tsne.legend.border_line_width = 3
    plot_tsne.legend.border_line_color = "black"
    plot_tsne.legend.border_line_alpha = 1
    plot_tsne.legend.location = "top_right"
    plot_tsne.legend.background_fill_color = "beige"
    plot_tsne.legend.background_fill_alpha = 0.5
    return plot_tsne

In [272]:
#output_file("graphs/vegetables_perfumes.html")
vegetables_perfumes = ["šim-gig", "hi{sar}"]
words_d = word_categories(model_t, vegetables_perfumes, 35)
p2 = tsne_bokeh(model_t, words_d, vegetables_perfumes, "12pt")
#save(p2)
show(p2)

In [ ]:
central_bureau = [cun("na-sa₆"), cun("ab-ba-sa₆-ga"), cun("in-ta-e₃-a")]
c_b = word_categories(model_c, central_bureau, 20)
p1 = tsne_bokeh(model_c, c_b, central_bureau, "20pt")
save(p1)
show(p1)

In [ ]:
output_file("graphs/vegetables_perfumes_l.html")
vegetables_perfumes = ["šimgig[tree]N", "hiz[vegetable]N"]
words_d = word_categories(model_l, vegetables_perfumes, 35)
p2 = tsne_bokeh(model_l, words_d, vegetables_perfumes, "12pt")
save(p2)
show(p2)

In [ ]:
model_c.wv.most_similar(cun("ab-ba-sa₆-ga"), topn = 20)

In [ ]:
naramili = cun("na-ra-am-i₃-li₂")

In [ ]:
model_c.wv.most_similar(naramili)

In [ ]:
model_c.wv.most_similar(cun("lugal-iti-da"))

In [ ]:
babati_abbasaga = [cun("ba-ba-ti"), cun("ab-ba-sa₆-ga")]
words_d = word_categories(model_c, babati_abbasaga, 10)
p2 = tsne_bokeh(model_c, words_d, babati_abbasaga, "20pt")
show(p2)

In [ ]:
p_d = ["ab-ba-sa₆-ga", "u₂-ta₂-mi-šar-ra-am", "a-hu-we-er", "tu-ra-am-d-da-gan"]

In [ ]:
p_d = ["šu-{d}idim", "šu-kab-ta", "šu-{d}kab₂-ta₂", "šu-{d}kab-ta₂"]

In [ ]:
for item in p_d: 
    print(item in model_t.wv.vocab)

In [ ]:
pd = [model_c.wv[p] for p in p_d]

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [ ]:
Z = linkage(pd, 'ward')

In [ ]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

In [ ]:
c, coph_dists = cophenet(Z, pdist(pd))
c

In [ ]:
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
def dendro(model, word, topn=10): 
    words = model.wv.most_similar(word, topn = topn)
    words = [w[0] for w in words]
    words.append(word)
    vectors = [model.wv[p] for p in words]
    l = linkage(vectors, method='complete', metric='seuclidean')

# calculate full dendrogram
    plt.figure(figsize=(25, 10))
    plt.title('Hierarchical Clustering Dendrogram')
    plt.ylabel('word')
    plt.xlabel('distance')

    dendrogram(
        l,
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=16.,  # font size for the x axis labels
        leaf_label_func=lambda v: str(words[v])
        )
    plt.show()

In [ ]:
dendro(model_t, "ha-bu-um")

In [13]:
model_c.wv.most_similar(positive=["𒊺", "𒆬𒄀"], negative=["𒃻"])

[('𒄭𒄊𒀀', 0.9438665509223938),
 ('𒋛𒁲', 0.9368212223052979),
 ('𒃻𒉿', 0.9308703541755676),
 ('𒀀𒁉𒍝', 0.9291903972625732),
 ('𒆬𒌓', 0.9288144111633301),
 ('𒐈𒅅', 0.9267836213111877),
 ('𒋗𒄥', 0.9232181906700134),
 ('𒌈𒁾', 0.9228167533874512),
 ('𒍝𒄵', 0.9221596121788025),
 ('𒄭𒁕', 0.9183017015457153)]

In [99]:
nakabtum_a = [cun("a-hu-ni"), cun("d-šul-gi-a-a-mu"), cun("šu-ma-ma"), cun("zu-ba-ga")]

In [122]:
for name in directors: 
    print(name in model_c.wv.vocab)

True
True
True


In [101]:
nakabtum_b = [cun("lu₂-dingir-ra"), cun("a-hu-we-er"), cun("igi-d-en-lil₂-še₃")]

In [105]:
royal_court = [cun("u₂-ta₂-mi-šar-ra-am"), cun("tu-ra-am-d-da-gan"), cun("tah-ša-tal")]

In [107]:
tummal = [cun("en-dingir-mu")]

In [109]:
enlila = [cun("d-en-lil₂-la₂")]

In [115]:
urkununa = [cun("ur-kug-nun-na")]

In [113]:
nalu = [cun("na-lu₅")]

In [117]:
central = [cun("ab-ba-sa₆-ga"), cun("in-ta-e₃-a"), cun("na-sa₆")]

In [121]:
directors = [cun("na-ra-am-i₃-li₂"), cun("lugal-iti-da"), cun("d-šara₂-kam")]

In [158]:
offices = [nakabtum_a, nakabtum_b, enlila, nalu, 
           urkununa, central, directors]
office_names = ["Nakabtum A", "Nakabtum B", "Enlila", "Nalu", "Urkununa", "Central Bureau", "Directors"]
p = figure(plot_width=900, plot_height=900) 

In [160]:
tokens = []
labels = []
categories = []
legend = []
for idx, office in enumerate(offices): 
    for name in office: 
        tokens.append(model_c.wv[name])
        labels.append(name)
        categories.append(idx)
        legend.append(office_names[idx])

[0, 0, 0, 0, 1, 1, 1, 2, 3, 4, 5, 5, 5, 6, 6, 6]

In [184]:
from bokeh.palettes import Category20
c = all_palettes["Spectral"][len(offices)]
colors = [c[category] for category in categories]

In [185]:
tsne = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
tsne_embedding = tsne.fit_transform(tokens)
tsne_embedding = pd.DataFrame(tsne_embedding, columns=['x','y'])
tsne_embedding["color"] = colors
tsne_embedding["labels"] = labels
tsne_embedding["legend"] = legend

In [186]:
source = ColumnDataSource(
        data=dict(
        x = tsne_embedding.x,
        y = tsne_embedding.y,
        colors = tsne_embedding.color,
        labels=tsne_embedding.labels,
        legend = tsne_embedding.legend
        )
    )

In [187]:
l = LabelSet(x='x', y='y', text='labels', level='glyph',
              x_offset=5, y_offset=5, source=source, render_mode='canvas', 
             text_font_size="20pt", text_font="CuneiformComposite")

plot_tsne = figure(plot_width=900, plot_height=900) #, tools=tools_tsne, title='Papers')
plot_tsne.circle('x', 'y', size=7, fill_color='colors', 
                  line_alpha=0, line_width=0.01, source=source, legend="legend")
plot_tsne.add_layout(l)
plot_tsne.legend.border_line_width = 3
plot_tsne.legend.border_line_color = "black"
plot_tsne.legend.border_line_alpha = 1
plot_tsne.legend.location = "top_left"
plot_tsne.legend.background_fill_color = "beige"
plot_tsne.legend.background_fill_alpha = 0.5

In [188]:
show(plot_tsne)

In [183]:
all_palettes["Spectral"][6]

['#3288bd', '#99d594', '#e6f598', '#fee08b', '#fc8d59', '#d53e4f']

In [189]:
with open("output/utf8_lemm_d.p", 'rb') as f: 
    utf8_lemm = pickle.load(f)
utf8_lemm

{'𒁹': {'1',
  '1(BARIG)',
  '1(DIŠ)',
  '1(barig)',
  '1(barig@c)',
  '1(diš)',
  '1(diš@c)',
  '1-AŠ₂-ta',
  '1-a',
  '1-a-kam',
  '1-a-kam-ma',
  '1-a-ni',
  '1-am₃',
  '1-bi',
  '1-e',
  '1-gin₇',
  '1-ka',
  '1-kam',
  '1-kam-ma',
  '1-kam-ma-am₃',
  '1-kam-ma-x',
  '1-kam₂',
  '1-la₂-kam',
  '1-ra',
  '1-ta',
  '1-ta-am₃',
  '1-še₃',
  'DIŠ',
  'ana',
  'barig@c',
  'diš',
  'diš@c',
  'diš[one]NU',
  'ge[blow]N',
  'ge₃',
  'makkaš',
  'makkaš[lamentation]N',
  'nigida',
  'santak₄',
  'saŋtak[triangle]N',
  'tal[clamour]N',
  'tal₄',
  'til₅',
  'ukitₓ(DIŠ)'},
 '𒊺': {'ašnan[grain]N',
  'eš[shrine]N',
  'niga',
  'niga[fattened]V/i',
  'sa[equal]V/t',
  'sag₁₃',
  'u[grain]N',
  'ŠE',
  'še',
  'še[barley]N',
  'še[cvne]N',
  'še[fire]N',
  'še[that]N',
  'šeg[agree]V/t',
  'šeg[voice]N'},
 '𒁀𒈜': {'ba-LUL', 'ba-lul', 'ba-lu₅', 'pa₂-lu₅'},
 '𒉄': {'NAGAR',
  'alla',
  'bulug[needle]N',
  'bulug₄',
  'e₁₀',
  'haya',
  'illa',
  'nagar',
  'nagar[carpenter]N',
  'niŋinₓ(NAGAR)'},
 '

In [191]:
utf8_lemm[cun("za-bir")]

{'zaʾellaŋ[bead]N'}

In [194]:
nakabtum_a

['𒀀𒄷𒉌', '𒀭𒂄𒄀𒀀𒀀𒈬', '𒋗𒈠𒈠', '𒍪𒁀𒂵']

In [200]:
wild_animals = [cun("ur-mah"), cun("muš"), cun("az"), cun("ka₅-a"), cun("šeg₉"), cun("maš-da₃"), cun("šeg₉-bar"), 
                cun("dara₄"), cun("am"), cun("am-si"), cun("am-si-kur-ra")]

In [204]:
domestic_animals = [cun("udu"), cun("gud"), cun("u₈"), cun("ab₂"), cun("anše"), cun("eme₅"), cun("sila₄"), cun("kir₁₁"), cun("amar"), 
                  cun("uz₃"), cun("maš₂") ]

In [234]:
animals = wild_animals.copy()
animals.extend(domestic_animals)

In [235]:
animal_like = model_c.wv.most_similar(animals, topn=20)
animal_like = [word for word, similarity in animal_like]

In [236]:
animals.extend(animal_like)

In [237]:
x = [model_c.wv.similarity(cun("ur-mah"), word) for word in animals]
y = [model_c.wv.similarity(cun("udu"), word) for word in animals]

In [238]:
df = pd.DataFrame({"x" : x, "y": y, "labels" : animals})

In [240]:
source = ColumnDataSource(
        data=dict(
        x = df.x,
        y = df.y,
        labels=df.labels,
        )
    )

In [261]:
lab = LabelSet(x='x', y='y', text='labels', level='glyph',
              x_offset=5, y_offset=5, source=source, render_mode='canvas', 
             text_font_size="15pt", text_font="CuneiformComposite")
p3 = figure(plot_width=900, plot_height=900) #, tools=tools_tsne, title='Papers')
p3.circle('x', 'y', size=7, fill_color='red', 
                  line_alpha=0, line_width=0.01, source=source)
left, right, bottom, top = .8, 1.01, .8, 1.01
p3.x_range=Range1d(left, right)
p3.y_range=Range1d(bottom, top)
p3.add_layout(lab)
p3.line([0, 1], [0, 1], line_width=2)

bokeh.models.renderers.GlyphRenderer(
    id='5459f7e5-8a73-435e-94db-c1754a8ffa88',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='510f4c9e-c470-4227-90e3-d5b2469ca750',
        callback=None,
        data={'x': [0, 1], 'y': [0, 1]},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        selected=bokeh.models.selections.Selection(
            id='bd39fc11-2bdf-4d03-9a2a-fdf0736d1f22',
            indices=[],
            js_event_callbacks={},
            js_property_callbacks={},
            line_indices=[],
            multiline_indices={},
            name=None,
            subscribed_events=[],
            tags=[]),
        selection_policy=bokeh.models.selections.UnionRenderers(
            id='85429a2d-c2ba-4f1d-8986-035d950c7cc5',
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    glyph=bokeh.models.glyphs.Line(
        id='e09b60af-2d65-4492-a0d8-a8f0dfa2b968',
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha=1.0,
        line_cap='butt',
        line_color='#1f77b4',
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width=2,
        name=None,
        subscribed_events=[],
        tags=[],
        x={'field': 'x'},
        y={'field': 'y'}),
    hover_glyph=None,
    js_event_callbacks={},
    js_property_callbacks={},
    level='glyph',
    muted=False,
    muted_glyph=None,
    name=None,
    nonselection_glyph=bokeh.models.glyphs.Line(
        id='814b2ec8-f95f-459f-b7e1-059b69bceea6',
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha=0.1,
        line_cap='butt',
        line_color='#1f77b4',
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width=2,
        name=None,
        subscribed_events=[],
        tags=[],
        x={'field': 'x'},
        y={'field': 'y'}),
    selection_glyph=None,
    subscribed_events=[],
    tags=[],
    view=bokeh.models.sources.CDSView(
        id='baf0db3c-62a8-4096-9ae5-59d0df0e0ab5',
        filters=[],
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        source=bokeh.models.sources.ColumnDataSource(
            id='510f4c9e-c470-4227-90e3-d5b2469ca750',
            callback=None,
            data={'x': [0, 1], 'y': [0, 1]},
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            selected=bokeh.models.selections.Selection(
                id='bd39fc11-2bdf-4d03-9a2a-fdf0736d1f22',
                indices=[],
                js_event_callbacks={},
                js_property_callbacks={},
                line_indices=[],
                multiline_indices={},
                name=None,
                subscribed_events=[],
                tags=[]),
            selection_policy=bokeh.models.selections.UnionRenderers(
                id='85429a2d-c2ba-4f1d-8986-035d950c7cc5',
                js_event_callbacks={},
                js_property_callbacks={},
                name=None,
                subscribed_events=[],
                tags=[]),
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    visible=True,
    x_range_name='default',
    y_range_name='default')

In [262]:
show(p3)

In [273]:
model_c

In [275]:
model_c.wv[cun("lugal")]

array([-0.44070795,  0.26555952,  2.5707364 ,  0.65769166,  1.4425119 ,
       -0.06138688,  1.3786849 , -1.4272784 , -0.2576735 , -2.6877482 ,
        0.15667441,  0.16343863, -0.8066997 , -0.30045176,  0.8043375 ,
       -2.065986  , -1.6161305 , -1.2269938 ,  1.8158962 ,  2.5451877 ,
       -0.58327955, -1.7063795 , -1.0180199 , -0.08752416, -0.02073782,
       -0.5937808 , -0.73376256, -1.4432474 ,  0.7808402 ,  1.2753896 ,
        0.1282989 , -0.50348336, -2.0970244 , -0.59052765, -0.46878287,
       -3.8291085 , -0.04018303, -2.1253262 , -1.035125  ,  1.5594889 ,
        2.5278628 ,  0.08087446, -0.05416838, -1.4248613 ,  0.00896834,
       -1.8636543 ,  1.0655923 ,  0.05885202,  0.27961951, -1.0163795 ,
        1.7294538 ,  0.62265867,  0.25415674,  0.78661114,  0.702546  ,
        0.44479975, -1.4940377 ,  0.7417113 ,  0.13840322,  1.5386573 ,
        0.82679415,  0.53430897, -2.95984   , -0.09329528,  2.2792082 ,
        0.36553383, -0.06652412,  1.2696545 ,  0.23809193, -1.10